In [3]:
import os
import pandas as pd
import json
import biom
import qiime2
from tqdm import tqdm
from qiime2.plugins import feature_table, diversity, taxa
from skbio import DistanceMatrix
from skbio.stats.distance import anosim
from skbio.stats.ordination import pcoa

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, 'results')
merged_dir = os.path.join(results_dir, 'merged')
beta_dir = os.path.join(merged_dir, 'beta')

def data_format(d):
    if d == 0.0 or d == -0.0:
        return '0'
    else:
        return str(d)

tax_list = ['', '', 'p__', 'c__', 'o__', 'f__', 'g__', 's__']
sample_metadata = qiime2.Metadata.load(os.path.join(data_dir, 'sample_metadata.txt'))
merged_taxonomy_class = qiime2.Artifact.load(os.path.join(merged_dir, 'merged_taxonomy_class.qza'))
merged_table_clean = qiime2.Artifact.load(os.path.join(merged_dir, 'merged_table_clean.qza'))
merged_rep_seqs = qiime2.Artifact.load(os.path.join(merged_dir, 'merged_rep_seqs.qza'))
rooted_tree = qiime2.Artifact.load(os.path.join(merged_dir, 'rooted_tree.qza'))

In [4]:
# beta diversity

beta_dict = dict()

braycurtis_distance_matrix = diversity.pipelines.beta(table = merged_table_clean, 
                                                    metric = 'braycurtis', 
                                                    n_jobs= 'auto').distance_matrix.view(DistanceMatrix)
group_list = sample_metadata.to_dataframe().loc[list(braycurtis_distance_matrix.ids), 'BodySite'].tolist()
pcoa_res = pcoa(braycurtis_distance_matrix,number_of_dimensions=2)
stat_significance = anosim(braycurtis_distance_matrix, group_list, permutations=999)
p_value = round(stat_significance['p-value'],2)
if p_value == 0:
    p_value = '<0.01'
else:
    p_value = '= ' + str(p_value)
stat = 'ANOSIM: ' + 'R = ' + data_format(round(stat_significance['test statistic'],2)) + ' P ' + p_value
beta_df = pcoa_res.samples
beta_df = beta_df.round(2)
beta_df['BodySite'] = group_list
beta_df.to_csv(os.path.join(beta_dir, 'braycurtis.csv'), index = False)
beta_dict['braycurtis'] = {
    'pc1(%)': int(round(pcoa_res.proportion_explained['PC1'], 2) * 100),
    'pc2(%)': int(round(pcoa_res.proportion_explained['PC2'], 2) * 100),
    'stats': stat
}

jaccard_distance_matrix = diversity.pipelines.beta(table = merged_table_clean, 
                                                metric = 'jaccard',
                                                n_jobs= 'auto').distance_matrix.view(DistanceMatrix)
group_list = sample_metadata.to_dataframe().loc[list(jaccard_distance_matrix.ids), 'BodySite'].tolist()
pcoa_res = pcoa(jaccard_distance_matrix,number_of_dimensions=2)
stat_significance = anosim(jaccard_distance_matrix, group_list, permutations=999)
p_value = round(stat_significance['p-value'],2)
if p_value == 0:
    p_value = '<0.01'
else:
    p_value = '= ' + str(p_value)
stat = 'ANOSIM: ' + 'R = ' + data_format(round(stat_significance['test statistic'],2)) + ' P ' + p_value
beta_df = pcoa_res.samples
beta_df = beta_df.round(2)
beta_df['BodySite'] = group_list
beta_df.to_csv(os.path.join(beta_dir, 'jaccard.csv'), index = False)
beta_dict['jaccard'] = {
    'pc1(%)': int(round(pcoa_res.proportion_explained['PC1'], 2) * 100),
    'pc2(%)': int(round(pcoa_res.proportion_explained['PC2'], 2) * 100),
    'stats': stat
}

unweighted_unifrac_distance_matrix = diversity.pipelines.beta_phylogenetic(table = merged_table_clean, 
                                                                        phylogeny = rooted_tree, 
                                                                        metric = 'unweighted_unifrac',
                                                                        threads = 'auto').distance_matrix.view(DistanceMatrix)
group_list = sample_metadata.to_dataframe().loc[list(unweighted_unifrac_distance_matrix.ids), 'BodySite'].tolist()
pcoa_res = pcoa(unweighted_unifrac_distance_matrix, number_of_dimensions=2)
stat_significance = anosim(unweighted_unifrac_distance_matrix, group_list, permutations=999)
p_value = round(stat_significance['p-value'],2)
if p_value == 0:
    p_value = '<0.01'
else:
    p_value = '= ' + str(p_value)
stat = 'ANOSIM: ' + 'R = ' + data_format(round(stat_significance['test statistic'],2)) + ' P ' + p_value
beta_df = pcoa_res.samples
beta_df = beta_df.round(2)
beta_df['BodySite'] = group_list
beta_df.to_csv(os.path.join(beta_dir, 'unweighted_unifrac.csv'), index = False)
beta_dict['unweighted_unifrac'] = {
    'pc1(%)': int(round(pcoa_res.proportion_explained['PC1'], 2) * 100),
    'pc2(%)': int(round(pcoa_res.proportion_explained['PC2'], 2) * 100),
    'stats': stat
}

weighted_unifrac_distance_matrix = diversity.pipelines.beta_phylogenetic(table = merged_table_clean, 
                                                                        phylogeny = rooted_tree, 
                                                                        metric = 'weighted_unifrac',
                                                                        threads = 'auto').distance_matrix.view(DistanceMatrix)
group_list = sample_metadata.to_dataframe().loc[list(weighted_unifrac_distance_matrix.ids), 'BodySite'].tolist()
pcoa_res = pcoa(weighted_unifrac_distance_matrix, number_of_dimensions=2)
stat_significance = anosim(weighted_unifrac_distance_matrix, group_list, permutations=999)
p_value = round(stat_significance['p-value'],2)
if p_value == 0:
    p_value = '<0.01'
else:
    p_value = '= ' + str(p_value)
stat = 'ANOSIM: ' + 'R = ' + data_format(round(stat_significance['test statistic'],2)) + ' P ' + p_value 
beta_df = pcoa_res.samples
beta_df = beta_df.round(2)
beta_df['BodySite'] = group_list
beta_df.to_csv(os.path.join(beta_dir, 'weighted_unifrac.csv'), index = False)
beta_dict['weighted_unifrac'] = {
    'pc1(%)': int(round(pcoa_res.proportion_explained['PC1'], 2) * 100),
    'pc2(%)': int(round(pcoa_res.proportion_explained['PC2'], 2) * 100),
    'stats': stat
}
with open(os.path.join(beta_dir, 'beta_stats.json'), 'w') as file:
    json.dump(beta_dict, file)

print('STEP 7  Done!')

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/gaoyuze/data/0036ba6b-337e-4a6e-b5f4-0309d2b4dc06/data/feature-table.biom -t /tmp/qiime2/gaoyuze/data/73382eed-be36-45ce-b307-2c12fafee5a4/data/tree.nwk -m unweighted -o /tmp/q2-LSMatFormat-kr1a8706

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /tmp/qiime2/gaoyuze/data/0036ba6b-337e-4a6e-b5f4-0309d2b4dc06/data/feature-table.biom -t /tmp/qiime2/gaoyuze/data/73382eed-be36-45ce-b307-2c12fafee5a4/data/tree.nwk -m weighted_unnormalized -o /tmp/q2-LSMatFormat-0chyvitg

STEP 7  Done!


/home/gaoyuze/anaconda3/envs/qiime2-2023.7/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.004622665718157072 and the largest is 0.4339928155624625.
  warn(
